# load dask cluster with data
load a parquet dataset into a dask cluster

In [ ]:
# nuclio: ignore
import nuclio

In [ ]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import json
import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import List, Optional

def load_dask(
    context: MLClientCtx,
    src_data: str,
    dask_key: str = "dask_df",
    inc_cols: Optional[List[str]] = None,
    index_cols: Optional[List[str]] = None,
    dask_persist: bool = True
) -> None:
    """Load dataset into an existing dask cluster
    
    dask jobs define the dask client parameters at the job level, this method will raise an error if no client is detected.
    
    :param context:         the function context
    :param src_data:        url of the data file or partitioned dataset as either
                            artifact DataItem, string, or path object (similar to 
                            pandas read_csv)
    :param dask_key:        destination key of data on dask cluster and artifact store
    :param inc_cols:        include only these columns (very fast)
    :param index_cols:      list of index column names (can be a long-running process)
    """
    if hasattr(context, "dask_client"):
        dask_client = context.dask_client
        print(dask_client)
    else:
        raise Exception("a dask client was not found in the execution context")
    
    context.logger.info(dask_client)

    # need utility: many functions use this if pq or csv, but could be dir with pq partitioned 
    # dataset
    src_data = str(src_data)
    if isinstance(src_data, string):
        if os.path.isdir(src_data) or src_data.endswith("pq") or src_data.endswith("parquet"):
            df = dd.read_parquet(src_data)
        elif src_data.endswith("csv"):
            df = dd.read_csv(src_data)

    if persist and context:
        df = dask_client.persist(df)
        dask_client.publish_dataset(dask_key=df)
        context.dask_client = dask_client
        
        # share the scheduler
        filepath = os.path.join(target_path, "scheduler.json")
        dask_client.write_scheduler_file(filepath)
        context.log_artifact("scheduler", target_path=filepath)
        
        print(df.head())


In [ ]:
# nuclio: end-code

### save

In [ ]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dask", kind="dask", with_doc=True,
                      handler=load_dask, image="mlrun/ml-dask:0.4.6")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dask"
fn.spec.description = "load dask cluster with data"
fn.metadata.categories = ["fileutils", "distributed"]
fn.spec.image_pull_policy = "Always"
fn.metadata.labels = {"author": "yjb"}
fn.spec.remote = True
fn.spec.replicas = 4 
fn.spec.max_replicas = 4
fn.spec.service_type = 'NodePort'
fn.spec.image = 'daskdev/dask:2.9.1'

fn.save()
fn.export("function.yaml")

### test

In [ ]:
from mlrun import import_function, mount_v3io, NewTask

func = import_function("hub://load_dask").apply(mount_v3io())

task_params = {
    "name":        "tasks load dask cluster with data",
    "params" : {
        "src_data"    : "/User/artifacts/higgs.parquet",
        "persist"     : True,
        "dask_key"    : "dask_df"}}

run = func.run(handler=load_dask, params=task_params, artifact_path="/User/artifacts")

In [ ]:
func.status.to_dict()

### our cluster

Let"s load the scheduler file into a cluster in this notebook:

In [ ]:
import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

In [ ]:
%%time 
client = Client('tcp://mlrun-load-dask-a647e320-1.default-tenant:8786')

In [ ]:
%%time 
df = client.get_dataset("dask_df")

In [ ]:
%%time 
df.shape[0].compute()

In [ ]:
client.nbytes(summary=False)